In [6]:
import re
import pandas as pd
import sqlite3
def read_select_rules(column_name):
    query = f"""
    SELECT {column_name} FROM T3_table
    WHERE Type = 'Select' AND [OD rules] IS NOT NULL AND [OD rules] LIKE '%Assertion%' """
    conn = sqlite3.connect('T3.db')
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    conn.close()
    return [row[0] for row in data if row[0] is not None]
components = read_select_rules('Component')
project_name = read_select_rules('[Project name]')
version = read_select_rules('Version')
date = read_select_rules('Date')
owner = read_select_rules('Owner')
OD_rules = read_select_rules('[OD rules]')
OD_rules = [re.sub(r'\bSBB\w*\b', '', rule).strip() for rule in OD_rules]
OD_rules = [re.sub(r'[()]', '', rule).strip() for rule in OD_rules]
rules = read_select_rules('rules')
newrules = [line.split(']')[0] for line in rules]
pattern = r'([✔✘])\s*(.*)'
data = []
section_id = 1
for section in OD_rules:
    items = re.split(r'\s*\|\|\s*|\s*&&\s*|\s*\(\s*|\s*\)\s*', section)
    section_data = []
    for item in items:
        match = re.match(pattern, item)
        if match:
            status, name = match.groups()
            section_data.append((section_id, status, name))
    if section_data:
        data.extend(section_data)
    section_id += 1
df = pd.DataFrame(data, columns=['Section_id', 'Status', 'name'])
print(df)

      Section_id Status                                               name
0              1      ✔                   9225 Rear Photocataly fan for TW
1              1      ✔  TW 8025 Front system fan , if ✔ RTX3050 4GB G6...
2              1      ✔                         RTX3060 12GB G6 192b H+3DP
3              1      ✔                        RX6500XT 4GB G6 64b H+DP HP
4              1      ✔                       GTX1650 SP 4GB G6 128b 2DP+H
...          ...    ...                                                ...
3463        1224      ✔                             2TB 7200RPM SATA3 3.5"
3464        1224      ✔                  M.2 2242 128G PCIe NVMe SSD - TLC
3465        1224      ✔         M.2 2280 256G PCIe NVMe OPAL 2.0 SSD - TLC
3466        1224      ✔         M.2 2280 512G PCIe NVMe OPAL 2.0 SSD - TLC
3467        1224      ✔                 M.2 2280  512G PCIe NVMe SSD - QLC

[3468 rows x 3 columns]


In [7]:
import sqlite3 
from py2neo import Graph, Node, Relationship
g = Graph('neo4j://localhost:7687', auth = ('neo4j', 'ys1203303'))
g.run("MATCH (n) DETACH DELETE n") 
for i in range(len(components)):
        node1 = Node(
            "parent",
            Name = newrules[i],
            Component = components[i],
            originalRule = rules[i],
            Type = "Select",
            ProjectName = project_name[i],
            Version = version[i],
            owner = owner[i], 
            date = date[i],
            ruleIndex = OD_rules[i])
        g.create(node1) 
        section_df = df[df['Section_id'] == i + 1] 
        print(section_df) 
        for s, ss in section_df.iterrows():
            node2 = Node(
                "son", 
                Name = ss['name'],
                Component = components[i],
                originalRule = rules[i],
                Type = "Select",
                ProjectName = project_name[i],
                Version = version[i],
                owner = owner[i], 
                date = date[i],
                ruleIndex = OD_rules[i])
            g.create(node2)
            if ss['Status'] == '✔':
                relationship = Relationship(node2, "should", node1) 
            else:
                relationship = Relationship(node2, "should not", node1) 
            g.create(relationship) 

   Section_id Status                                               name
0           1      ✔                   9225 Rear Photocataly fan for TW
1           1      ✔  TW 8025 Front system fan , if ✔ RTX3050 4GB G6...
2           1      ✔                         RTX3060 12GB G6 192b H+3DP
3           1      ✔                        RX6500XT 4GB G6 64b H+DP HP
4           1      ✔                       GTX1650 SP 4GB G6 128b 2DP+H
5           1      ✔                     GTX1660SP 6GB G6 192b DVI+DP+H
6           1      ✔                       RTX3050 8GB G6 128b H+3DP HP
    Section_id Status                                               name
7            2      ✘                  3.5’’ SATA3.0 2TB 7200 RPM/6Gb 4K
8            2      ✘                  3.5’’ SATA3.0 1TB 7200 RPM/6Gb 4K
9            2      ✘  3.5’’ SATA3.0 2TB 7200 RPM/6Gb 4K, if ✘  3.5’’...
10           2      ✘                  3.5’’ SATA3.0 2TB 7200 RPM/6Gb 4K
11           2      ✘     M.2 2242 128G Value PCIe Gen3x4 N